In [1]:
import numpy as np #Matrices
import os #Funciones del sistema
import six.moves.urllib as urllib #URL
import sys #Funciones del sistema
import tarfile #Leer y escribir archivos tar
import tensorflow as tf #Aprendizaje
import zipfile #Leer y escribir archivos zip 

from collections import defaultdict #Tipos de contenedores
from io import StringIO #Objetos tipo String
from matplotlib import pyplot as plt #Procesamiento gráfico
from PIL import Image #Soporte para formatos de imagenes

In [2]:
sys.path.append('C:\Models\models\research\object_detection') #Nos vamos a la ruta donde se debe ejecutar el script
from object_detection.utils import label_map_util #Etiquetas
from object_detection.utils import visualization_utils as vis_util #Objetos

import cv2 #OpenCV visión artificial

cap = cv2.VideoCapture(0)   # Capturar el contenido de la web cam 

In [3]:
# Modelo a descargar COCO
MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Ruta al gráfico de detección. Este es el modelo real que se utiliza para la detección de objetos.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# Lista de cadenas usadas para añadir la etiqueta correta para cada caja
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [4]:
# ## Descargar el modelo

opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())

OSError: [Errno socket error] [Errno 11001] getaddrinfo failed

In [ ]:
# ## Cargar el modelo de tensorflow a la memoria

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [ ]:
# Cargar el mapa de etiquetas
# El mapa apunta a categorías, la red convolucional retorna un indice (int) de dicha categoria. 

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [ ]:
# ## Función para las matrices

def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)



In [ ]:
#Detección
# Si se desean añadir imagenes para probar, se agregan a la ruta: TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]  # change this value if you want to add more pictures to test

# Tamaño en pulgadas de las imagenes de salida
IMAGE_SIZE = (12, 8)

In [ ]:
#Realizamos un ciclo para que nos muestre las capturas en forma secuencial, o sea un video. 
with detection_graph.as_default():
    #Creamos un objeto de sesion para poder ejecutar las operaciones de Tensorflow
    with tf.Session(graph=detection_graph) as sess:
        while True:
            ret, image_np = cap.read()
            # Expande las dimensiones ya que el modelo espera la entrada : [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Cada caja representa una parte de la imagen donde fue detectado un objeto en particular
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Cada puntuacion representa que tanto se acerca a la definición de un objeto
            # Se muestra el porcentaje de aproximación a cada objeto junto con la etiqueta
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            # Detección actual
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Mostrar resultados de la detección
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
            #Mostrar la imagen en una ventana (video o imagen)
            cv2.imshow('object detection', cv2.resize(image_np, (800, 600)))
            #Tecla para cerrar la ventana <<q>>
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break